# BentoML PyTorch TLDR Tutorial

Link to source code: https://github.com/bentoml/gallery/tree/main/TLDR/pytorch_tldr_demo/

Install required dependencies:

In [1]:
!python3 -m pip install --quiet -r requirements.txt

Import required libraries:

## Download and Extract Reddit TL;DR

In [2]:
import requests
from os.path import exists
import zipfile
import json
from tqdm import trange, tqdm

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
dataset_path = 'dataset/corpus-webis-tldr-17.zip'

if not exists(dataset_path):
    r = requests.get('https://zenodo.org/record/1043504/files/corpus-webis-tldr-17.zip?download=1')
    with open(dataset_path, 'wb') as f:
        f.write(r.content)
    with zipfile.ZipFile(dataset_path, 'r') as z:
        z.extractall('dataset/')

In [4]:
df = pd.read_json('dataset/corpus-webis-tldr-17.json', \
                  lines=True, chunksize=100000)

## Begin processing data

In [5]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [6]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    lines = next(df)

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l] for l in \
             tqdm(zip(lines['content'], lines['summary']))]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
MAX_LENGTH = 260

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re ",
    
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [8]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('content', 'summary')
print(random.choice(pairs))

Reading lines...


100000it [00:35, 2846.61it/s]


Read 100000 sentence pairs
Trimmed to 2841 sentence pairs
Counting words...
Counted words:
content 19584
summary 6463
['i am going to expand on your comment to clarify it is a combination of both but depending on where you are coming from changes how you talk about it . every single top player knows the odds and how to make the right decisions based on the math . but if you simply fallow this and everyone else is then you are all just leaving it to chance . so at a pro level picking the right times to push your odds based on how you are reading the other players sets you apart . in a game with non professionals most don t really understand the math . so knowing these things gives you an edge because they think it is all about the players . and they will generally try and overdo it and they ll put themselves out of the game . on the flip side it is hard to bluff people when they think their shit hand is good and similarly it is hard to semi bluff when they don t see the chance of failur

## Define the model

First let's define a simple PyTorch network

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Training and Saving the model

Then we define a simple PyTorch network and some helper functions

In [12]:
def indexesFromSentence(lang, sentence):
    indices = []
    for word in sentence.split(' '):
        if word in lang.word2index:
            # lang.addWord(word)
            indices.append(lang.word2index[word])
    return indices


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [13]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [14]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [15]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
            # input_lang, output_lang, pairs_2 = prepareData('content', 'summary')
            # training_pairs = [tensorsFromPair(random.choice(pairs_2))
            #       for i in range(n_iters)]

    showPlot(plot_losses)

### Cross Validation

We can do some cross validation and the results can be saved with the model as metadata


In [17]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [18]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [19]:
hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 6000, print_every=100)

1m 58s (- 116m 41s) (100 1%) 5.4868
3m 59s (- 115m 53s) (200 3%) 4.7123
5m 48s (- 110m 24s) (300 5%) 4.1829
7m 50s (- 109m 43s) (400 6%) 3.7229
9m 38s (- 106m 3s) (500 8%) 4.0344
11m 38s (- 104m 44s) (600 10%) 4.1306
13m 35s (- 102m 56s) (700 11%) 3.5692
15m 58s (- 103m 51s) (800 13%) 3.3792
18m 3s (- 102m 20s) (900 15%) 3.8816
20m 7s (- 100m 36s) (1000 16%) 3.9263
22m 8s (- 98m 35s) (1100 18%) 3.7119
24m 4s (- 96m 16s) (1200 20%) 3.6045
26m 2s (- 94m 10s) (1300 21%) 3.6248
27m 53s (- 91m 37s) (1400 23%) 3.8138
29m 57s (- 89m 51s) (1500 25%) 3.9532
31m 44s (- 87m 16s) (1600 26%) 3.8311
33m 42s (- 85m 16s) (1700 28%) 3.8689
35m 42s (- 83m 19s) (1800 30%) 3.2818
37m 38s (- 81m 12s) (1900 31%) 3.4302
39m 44s (- 79m 29s) (2000 33%) 3.6502
41m 49s (- 77m 39s) (2100 35%) 3.5135
43m 56s (- 75m 54s) (2200 36%) 4.0707
46m 5s (- 74m 8s) (2300 38%) 3.7621
48m 2s (- 72m 4s) (2400 40%) 3.6728
50m 20s (- 70m 28s) (2500 41%) 3.6468
52m 36s (- 68m 47s) (2600 43%) 3.6676
54m 47s (- 66m 57s) (2700 45%) 

In [20]:
evaluateRandomly(encoder1, attn_decoder1)

> i m going to have to get back to you on this i don t know for sure . i just treat them the same . edit research led me to this the differences between hypertrophic and keloid scars can be confusing . although they both have excessive collagen in common they act differently . for instance a hypertrophic scar forms soon after the skin has been lacerated while keloids may not begin forming for up to a year after the injury .the tissue structure also varies between the two scar types . collagen fiber formation in keloid scars actively spreads into the nearby healthy tissue while hypertrophic scars remain contained within the original boundary of the injury . the surrounding healthy tissue is not affected .
= they re the same though they can form at different times and one can spread . baby your piercing hole .
< you are a a you . <EOS>

> well his allergy is really strange . he doesn t just just swell up and die if he eats something with a spec of gluten in it . basically if he eats some

### saving the model

In [39]:
# Specify a path to save to
PATH = "summarizer.pt"

torch.save({
            'encoder1': encoder1,
            'attn_decoder1': attn_decoder1,
            }, PATH)

In [38]:
import bentoml

tag_encoder = bentoml.pytorch.save(
    "pytorch_tldr_encoder",
    encoder1,
)

tag_decoder = bentoml.pytorch.save(
    "pytorch_tldr_decoder",
    attn_decoder1,
)

AttributeError: 'SingletonFactory' object has no attribute 'register'

## Create a BentoML Service for serving the model

Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

Even though we have only one model, we can create as many api endpoints as we want. Here we create two end points `predict_ndarray` and `predict_image`

### load the model

In [40]:
# checkpoint = torch.load(PATH)

# encoder = checkpoint['encoder1']
# decoder = checkpoint['attn_decoder1']

In [41]:
import typing as t

import bentoml
from bentoml.io import Text


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


encoder = bentoml.pytorch.load_runner(
    "pytorch_tldr_encoder"
)

decoder = bentoml.pytorch.load_runner(
    "pytorch_tldr_decoder"
)

svc = bentoml.Service(
    name="pytorch_tldr_demo",
    runners=[
        encoder,
        decoder,
    ],
)


@svc.api(input=Text(), output=Text())
def summarize(input_arr: t.List[str]) -> t.List[str]:
    input_arr = list(map(normalizeString, input_arr))
    enc_arr = encoder.run_batch(input_arr)
    res = decoder.run_batch(enc_arr)
    return res[0]['generated_text']

TypeError: get() takes 1 positional argument but 2 were given

Start a dev model server to test out the service defined above

In [ ]:
!python3 -m bentoml serve service.py:svc

[13:51:28] INFO     Starting development BentoServer from "service.py:svc"      
[13:51:36] INFO     Service imported from source:                               
                    bentoml.Service(name="pytorch_tldr_demo",                   
                    import_str="service:svc", working_dir="/Users/spence/Documen
                    ts/GitHub/gallery/pytorch_seq2seq")                         
[13:51:36] INFO     Started server process [17881]                  ]8;id=341799;file:///Users/spence/Library/Python/3.8/lib/python/site-packages/uvicorn/server.py\server.py]8;;\:]8;id=96038;file:///Users/spence/Library/Python/3.8/lib/python/site-packages/uvicorn/server.py#82\82]8;;\
           INFO     Waiting for application startup.                    ]8;id=42783;file:///Users/spence/Library/Python/3.8/lib/python/site-packages/uvicorn/lifespan/on.py\on.py]8;;\:]8;id=171161;file:///Users/spence/Library/Python/3.8/lib/python/site-packages/uvicorn/lifespan/on.py#45\45]8;;\

## Build a Bento for distribution and deployment

In [29]:
bentoml.build(
    "service.py:svc",
    include=["*.py"],
    exclude=["tests/"],
    description="file:./README.md",
    python=dict(
        packages=["torch"],
    )
)

BentoMLException: API predict is already defined in Service pytorch_tldr_demo

Starting a dev server with the Bento build:

In [ ]:
!bentoml serve pytorch_tldr_demo:latest